# Projeto Covid - Situação Brasil
## Por: Michel Souza Santana
> Data: 21/01/2023

* Fonte: https://www.kaggle.com/datasets/shree1992/housedata

## Questão de Negócio
> Identificar o problema de negócio a ser resolvido

* Os mercados imobiliários, como os de Sydney e Melbourne, apresentam uma oportunidade interessante para os analistas de dados analisarem e preverem para onde os preços dos imóveis estão se movendo. A previsão dos preços dos imóveis está se tornando cada vez mais importante e benéfica. Os preços dos imóveis são um bom indicador tanto da condição geral do mercado quanto da saúde econômica de um país. Considerando os dados fornecidos, estamos lidando com um grande conjunto de registros de vendas de imóveis armazenados em um formato desconhecido e com problemas de qualidade de dados desconhecidos

## Entendimento do Negócio
> ...


## Coleta de Dados
> ...


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('archive/data.csv')
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


## Limpeza dos Dados
> ...


## Exploração dos Dados
> ...


## Modelagem dos Dados
> ...


## Algoritmos de Machine Laearning
> ...


## Avaliação do algoritimo
> ...


## Storytelling
> ...